In [1]:
import os
from urllib.request import urlopen
from requests import get
from urllib.parse import urlparse
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np

# Before

In [2]:
Before = pd.read_csv("Before_bike.csv")  

In [3]:
Before

,tripduration,starttime,stoptime,start station id,usertype,gender
0,393,2019-08-01 00:00:01.4680,2019-08-01 00:06:35.3780,531.0,Subscriber,2
1,627,2019-08-01 00:00:01.9290,2019-08-01 00:10:29.7840,274.0,Subscriber,2
2,1132,2019-08-01 00:00:04.0480,2019-08-01 00:18:56.1650,2000.0,Subscriber,1
3,1780,2019-08-01 00:00:04.1630,2019-08-01 00:29:44.7940,479.0,Subscriber,1
4,1517,2019-08-01 00:00:05.4580,2019-08-01 00:25:23.4550,3312.0,Subscriber,2
...,...,...,...,...,...,...
10485866,565,2020-01-30 14:50:26.6360,2020-01-30 14:59:52.0950,3359.0,Subscriber,1
10485867,410,2020-01-30 14:50:26.9530,2020-01-30 14:57:17.6620,358.0,Subscriber,2
10485868,159,2020-01-30 14:50:26.9660,2020-01-30 14:53:06.0540,515.0,Subscriber,1
10485869,117,2020-01-30 14:50:28.0260,2020-01-30 14:52:25.7980,3344.0,Subscriber,1


In [4]:
Before = Before[Before['gender'] != 0]

In [5]:
# 将秒转换为分钟
Before['tripduration_minutes'] = Before['tripduration'] / 60

# 创建三个分类的哑变量
Before['duration_category'] = pd.cut(Before['tripduration_minutes'], bins=[0, 15, 30, np.inf], labels=['within_15min', '15-30mins', 'above30mins'])
Before = pd.get_dummies(Before, columns=['duration_category'])
Before = pd.get_dummies(Before, columns=['usertype'])

/tmp/ipykernel_4261/3985718763.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Before['tripduration_minutes'] = Before['tripduration'] / 60
/tmp/ipykernel_4261/3985718763.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Before['duration_category'] = pd.cut(Before['tripduration_minutes'], bins=[0, 15, 30, np.inf], labels=['within_15min', '15-30mins', 'above30mins'])


In [6]:
Before

,tripduration,starttime,stoptime,start station id,gender,tripduration_minutes,duration_category_within_15min,duration_category_15-30mins,duration_category_above30mins,usertype_Customer,usertype_Subscriber
0,393,2019-08-01 00:00:01.4680,2019-08-01 00:06:35.3780,531.0,2,6.550000,True,False,False,False,True
1,627,2019-08-01 00:00:01.9290,2019-08-01 00:10:29.7840,274.0,2,10.450000,True,False,False,False,True
2,1132,2019-08-01 00:00:04.0480,2019-08-01 00:18:56.1650,2000.0,1,18.866667,False,True,False,False,True
3,1780,2019-08-01 00:00:04.1630,2019-08-01 00:29:44.7940,479.0,1,29.666667,False,True,False,False,True
4,1517,2019-08-01 00:00:05.4580,2019-08-01 00:25:23.4550,3312.0,2,25.283333,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
10485866,565,2020-01-30 14:50:26.6360,2020-01-30 14:59:52.0950,3359.0,1,9.416667,True,False,False,False,True
10485867,410,2020-01-30 14:50:26.9530,2020-01-30 14:57:17.6620,358.0,2,6.833333,True,False,False,False,True
10485868,159,2020-01-30 14:50:26.9660,2020-01-30 14:53:06.0540,515.0,1,2.650000,True,False,False,False,True
10485869,117,2020-01-30 14:50:28.0260,2020-01-30 14:52:25.7980,3344.0,1,1.950000,True,False,False,False,True


In [7]:
Before['gender'] = Before['gender'].replace({1: 'Male', 2: 'Female'})
Before = pd.get_dummies(Before, columns=['gender'])

In [8]:
Before['date'] = pd.to_datetime(Before['starttime']).dt.date

# 按日期对数据进行分组
daily_Before = Before.groupby('date').agg(
    total_trips=('tripduration_minutes', 'count'),  # 总出行量
    trips_within_15min=('duration_category_within_15min', 'sum'),  # 15分钟以内的出行量
    trips_15_to_30min=('duration_category_15-30mins', 'sum'),  # 15到30分钟的出行量
    trips_above30min=('duration_category_above30mins', 'sum'),  # 30分钟以上的出行量
    Customer_trips=('usertype_Customer', 'sum'),  # Customer的出行量
    Subscriber_trips=('usertype_Subscriber', 'sum'),  # Subscriber的出行量
    Female_trips=('gender_Female', 'sum')  # 女性出行量
)

# 计算15分钟以内、15到30分钟和30分钟以上、Subscriber 和 Customer、以及女性的比例
daily_Before['percentage_within_15min'] = daily_Before['trips_within_15min'] / daily_Before['total_trips']
daily_Before['percentage_15_to_30min'] = daily_Before['trips_15_to_30min'] / daily_Before['total_trips']
daily_Before['percentage_above30min'] = daily_Before['trips_above30min'] / daily_Before['total_trips']
daily_Before['percentage_Customer'] = daily_Before['Customer_trips'] / daily_Before['total_trips']
daily_Before['percentage_Subscriber'] = daily_Before['Subscriber_trips'] / daily_Before['total_trips']
daily_Before['percentage_Female'] = daily_Before['Female_trips'] / daily_Before['total_trips']
daily_Before['percentage_Male'] = 1 - daily_Before['Female_trips'] / daily_Before['total_trips']

In [9]:
daily_Before_pre = daily_Before[[
    'total_trips',
    'percentage_within_15min','percentage_15_to_30min','percentage_above30min','percentage_Customer','percentage_Subscriber','percentage_Female',
    'percentage_Male']]
daily_Before_pre.reset_index(inplace=True)
# daily_Before_pre['date'] = pd.to_datetime(daily_Before_pre['date'])

In [23]:
# daily_Before_pre['weekday'] = daily_Before_pre['date'].dt.weekday
# daily_Before_pre['is_weekday'] = daily_Before_pre['weekday'].apply(lambda x: 1 if x < 5 else 0)
# weekday (1) weekend (0)

/tmp/ipykernel_7860/3980279109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_Before_pre['weekday'] = daily_Before_pre['date'].dt.weekday
/tmp/ipykernel_7860/3980279109.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_Before_pre['is_weekday'] = daily_Before_pre['weekday'].apply(lambda x: 1 if x < 5 else 0)


In [10]:
daily_Before_pre

,date,total_trips,percentage_within_15min,percentage_15_to_30min,percentage_above30min,percentage_Customer,percentage_Subscriber,percentage_Female,percentage_Male
0,2019-08-01,82156,0.695470,0.234213,0.070317,0.080846,0.919154,0.259871,0.740129
1,2019-08-02,75731,0.681702,0.243229,0.075068,0.097939,0.902061,0.266337,0.733663
2,2019-08-03,65371,0.584403,0.294764,0.120833,0.204494,0.795506,0.319270,0.680730
3,2019-08-04,57344,0.615653,0.274920,0.109427,0.191633,0.808367,0.308664,0.691336
4,2019-08-05,75689,0.674946,0.247922,0.077131,0.095932,0.904068,0.267423,0.732577
...,...,...,...,...,...,...,...,...,...
178,2020-01-26,33599,0.732760,0.204411,0.062829,0.081877,0.918123,0.272984,0.727016
179,2020-01-27,44948,0.770335,0.185125,0.044540,0.030101,0.969899,0.228197,0.771803
180,2020-01-28,47167,0.780609,0.178854,0.040537,0.031314,0.968686,0.226726,0.773274
181,2020-01-29,45792,0.781425,0.178503,0.040073,0.029831,0.970169,0.228774,0.771226


In [11]:
daily_Before_pre.to_csv("before_pre.csv",index=False)

# After

In [2]:
After = pd.read_csv("After_bike.csv")

In [3]:
After

,tripduration,starttime,stoptime,start station id,usertype,gender
0,1062,2020-06-01 00:00:03.3720,2020-06-01 00:17:46.2080,3419,Customer,2
1,3810,2020-06-01 00:00:03.5530,2020-06-01 01:03:33.9360,366,Subscriber,0
2,1017,2020-06-01 00:00:09.6140,2020-06-01 00:17:06.8330,389,Customer,2
3,226,2020-06-01 00:00:12.1780,2020-06-01 00:03:58.8640,3255,Customer,0
4,1437,2020-06-01 00:00:21.2550,2020-06-01 00:24:18.9650,367,Customer,2
...,...,...,...,...,...,...
13880317,2045,2020-12-31 23:58:21.7040,2021-01-01 00:32:27.1570,526,Subscriber,1
13880318,288,2020-12-31 23:58:37.8400,2021-01-01 00:03:26.3250,3307,Subscriber,0
13880319,152,2020-12-31 23:58:52.8090,2021-01-01 00:01:25.6720,3510,Customer,0
13880320,654,2020-12-31 23:59:55.9550,2021-01-01 00:10:50.6050,4073,Subscriber,1


In [4]:
After = After[After['gender'] != 0]

In [5]:
# 将秒转换为分钟
After['tripduration_minutes'] = After['tripduration'] / 60

# 创建三个分类的哑变量
After['duration_category'] = pd.cut(After['tripduration_minutes'], bins=[0, 15, 30, np.inf], labels=['within_15min', '15-30mins', 'above30mins'])
After = pd.get_dummies(After, columns=['duration_category'])
After = pd.get_dummies(After, columns=['usertype'])

After['gender'] = After['gender'].replace({1: 'Male', 2: 'Female'})
After = pd.get_dummies(After, columns=['gender'])

/tmp/ipykernel_4499/2115794987.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  After['tripduration_minutes'] = After['tripduration'] / 60
/tmp/ipykernel_4499/2115794987.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  After['duration_category'] = pd.cut(After['tripduration_minutes'], bins=[0, 15, 30, np.inf], labels=['within_15min', '15-30mins', 'above30mins'])


In [6]:
After['date'] = pd.to_datetime(After['starttime']).dt.date

# 按日期对数据进行分组
daily_After = After.groupby('date').agg(
    total_trips=('tripduration_minutes', 'count'),  # 总出行量
    trips_within_15min=('duration_category_within_15min', 'sum'),  # 15分钟以内的出行量
    trips_15_to_30min=('duration_category_15-30mins', 'sum'),  # 15到30分钟的出行量
    trips_above30min=('duration_category_above30mins', 'sum'),  # 30分钟以上的出行量
    Customer_trips=('usertype_Customer', 'sum'),  # Customer的出行量
    Subscriber_trips=('usertype_Subscriber', 'sum'),  # Subscriber的出行量
    Female_trips=('gender_Female', 'sum')  # 女性出行量
)

# 计算15分钟以内、15到30分钟和30分钟以上、Subscriber 和 Customer、以及女性的比例
daily_After['percentage_within_15min'] = daily_After['trips_within_15min'] / daily_After['total_trips']
daily_After['percentage_15_to_30min'] = daily_After['trips_15_to_30min'] / daily_After['total_trips']
daily_After['percentage_above30min'] = daily_After['trips_above30min'] / daily_After['total_trips']
daily_After['percentage_Customer'] = daily_After['Customer_trips'] / daily_After['total_trips']
daily_After['percentage_Subscriber'] = daily_After['Subscriber_trips'] / daily_After['total_trips']
daily_After['percentage_Female'] = daily_After['Female_trips'] / daily_After['total_trips']
daily_After['percentage_Male'] = 1 - daily_After['Female_trips'] / daily_After['total_trips']

In [7]:
daily_After

,total_trips,trips_within_15min,trips_15_to_30min,trips_above30min,Customer_trips,Subscriber_trips,Female_trips,percentage_within_15min,percentage_15_to_30min,percentage_above30min,percentage_Customer,percentage_Subscriber,percentage_Female,percentage_Male
date,,,,,,,,,,,,,,
2020-06-01,46138,21706,14814,9618,8846,37292,15778,0.470458,0.321080,0.208462,0.191729,0.808271,0.341974,0.658026
2020-06-02,40581,18390,13307,8884,7274,33307,14131,0.453168,0.327912,0.218920,0.179246,0.820754,0.348217,0.651783
2020-06-03,25915,13332,7917,4666,3627,22288,8965,0.514451,0.305499,0.180050,0.139958,0.860042,0.345939,0.654061
2020-06-04,35301,16598,11025,7678,6161,29140,12717,0.470185,0.312314,0.217501,0.174528,0.825472,0.360245,0.639755
2020-06-05,29935,15967,8798,5170,4298,25637,10221,0.533389,0.293903,0.172708,0.143578,0.856422,0.341440,0.658560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,21190,14062,4907,2221,2364,18826,6550,0.663615,0.231571,0.104814,0.111562,0.888438,0.309108,0.690892
2020-12-28,34858,23436,8048,3374,3433,31425,10349,0.672328,0.230880,0.096793,0.098485,0.901515,0.296890,0.703110
2020-12-29,30292,21396,6548,2348,2390,27902,8624,0.706325,0.216163,0.077512,0.078899,0.921101,0.284696,0.715304


In [8]:
daily_After_pre = daily_After[[
    'total_trips',
    'percentage_within_15min','percentage_15_to_30min','percentage_above30min','percentage_Customer','percentage_Subscriber','percentage_Female',
    'percentage_Male']]
daily_After_pre.reset_index(inplace=True)
# daily_After_pre['date'] = pd.to_datetime(daily_After_pre['date'])

In [9]:
# daily_After_pre['weekday'] = daily_After_pre['date'].dt.weekday
# daily_After_pre['is_weekday'] = daily_After_pre['weekday'].apply(lambda x: 1 if x < 5 else 0)
# weekday (1) weekend (0)

/tmp/ipykernel_8586/4005703101.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_After_pre['weekday'] = daily_After_pre['date'].dt.weekday
/tmp/ipykernel_8586/4005703101.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_After_pre['is_weekday'] = daily_After_pre['weekday'].apply(lambda x: 1 if x < 5 else 0)


In [9]:
daily_After_pre.to_csv("after_pre.csv", index=False)